In [1]:
import glob
import uqer
import time
import joblib
import pickle
import warnings
import datetime
import numpy as np
import pandas as pd
import lightgbm as lgb
from uqer import DataAPI   #优矿api
from sklearn import linear_model
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from pandas.core.common import SettingWithCopyWarning
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)
client = uqer.Client(token='18266a7c0ac9f8cdbe00f9b2ecb65f42316a5f78d9cc22ebabcbd923593356e4')

127807@wmcloud.com 账号登录成功


In [2]:
def ticekrToStr(job_sum):
    job_sum = job_sum[job_sum.ticker<700000]
    job_sum.loc[job_sum.ticker<10,'temp']='00000'
    job_sum.loc[(job_sum.ticker<100)&(job_sum.ticker>=10),'temp']='0000'
    job_sum.loc[(job_sum.ticker<1000)&(job_sum.ticker>=100),'temp']='000'
    job_sum.loc[(job_sum.ticker<10000)&(job_sum.ticker>=1000),'temp']='00'
    job_sum.loc[job_sum.temp==job_sum.temp,'ticker'] = job_sum[job_sum.temp==job_sum.temp]['temp']+job_sum[job_sum.temp==job_sum.temp]['ticker'].astype(str)
    del job_sum['temp']
    job_sum['ticker'] = job_sum['ticker'].astype(str)
    return job_sum

def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

class ExcludeExtreme(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_median = None
        self.df_standard = None

    def fit(self, df_columnList):
        self.df_median = df_columnList.median()
        self.df_standard = df_columnList.apply(lambda x: x - self.df_median[x.name]).abs().median()
        return self

    def scaller(self, x):
        self.di_max = self.df_median[x.name] + 5 * self.df_standard[x.name]
        self.di_min = self.df_median[x.name] - 5 * self.df_standard[x.name]
        x = x.apply(lambda v: self.di_min if v < self.di_min else v)
        x = x.apply(lambda v: self.di_max if v > self.di_max else v)
        return x

    def transform(self, df_columnList):
        df_columnList = df_columnList.apply(self.scaller)
        return df_columnList

In [3]:
factors = pickle.load(open('pct1_cal/f_x_066', 'rb'))
extra_factors = ['volume', 'cachgPct', 'thecommittee', 'askVolume1',
       'bidVolume1', 'caQrr', 'caTr', 'OCVP1',
       'Open/vwap-1', 'Gap']
f_x = []
for ele in factors:
#gaopin_liangjia_FACTORS，minute_FACTORS,pyCov_FACTORS
    if ele[:5] in ['alpha','gaopi','py_co'] or ele[:8] == 'minute_F':
       f_x.append(ele)
f_x.extend(extra_factors)

In [4]:
f_index = ['ticker', 'tradeDate']
f_industry = pickle.load(open("pct1_cal/每日预测/f_industry", "rb"))
label_list = ['PCT5_rank', 'PCT2_rank', 'openclose_pct1_rank', 'openclose_pct1', 
'openclose_pct1_full', 'openclose_pct1_full_rank', 
'vwap_pct1_full', 'vwap_pct1_full_rank', 'askbid_pct1_rank', 'askbid_pct1']
f_y_list =  label_list[2:6]

In [47]:
df1 = pd.read_csv('pct1_cal/modified_alter_alphas_066_labels.csv')
label = pd.read_csv('labels/openclose_pct1_full.csv')
jihejingjia = pd.read_csv('pct1_cal/集合竞价因子.csv')
jihejingjia = jihejingjia.drop(columns='Unnamed: 0')
mevtransformer = ExcludeExtreme()
mevtransformer.fit(jihejingjia[extra_factors])
jihejingjia[extra_factors] = mevtransformer.transform(jihejingjia[extra_factors])
df1 = df1.merge(jihejingjia, on=['ticker', 'tradeDate'], how='left')

In [48]:
df1 = df1.merge(label, on=['ticker', 'tradeDate'], how='left')
df1 = df1[~df1.openclose_pct1_full_rank.isnull()]
df1 = df1[~df1.openclose_pct1_full.isnull()]
df1 = df1[~df1.openclose_pct1_rank.isnull()]
df1 = df1[~df1.openclose_pct1.isnull()]

In [7]:
data_source = 'full_Alter_066_modified_exta'
file_location = 'pct1_cal'
file_type = 'full'
model_list = ['LinearRegression', 'RidgeR', 'DecisionTreeR', 'XGBoostR', 'LGBMRegressor']
model_name = model_list[4]
target_types = ['r', 'c'] # 分类问题还是回归问题 r 回归问题 c 分类问题
target_type = target_types[0]

In [8]:
dates = df1.tradeDate.sort_values().unique()
epoch_ts = list(dates)
f_index = ['ticker', 'tradeDate']
#result_name = '{}_{}_{}_{}'.format(data_source, model_name, f_y, target_type)
#print(result_name)

update = 22 # 训练长度：22天
train_si = epoch_ts.index('2017-01-03') # included. '2017-01-03'
train_ei = epoch_ts.index('2019-01-02') # excluded. '2018-12-28'
test_si = epoch_ts.index('2019-01-02') # included. '2019-01-02'
test_ei = epoch_ts.index('2019-02-01') # excluded. '2019-01-31' '2019-02-01'
test_fi = len(epoch_ts) - 1 # excluded. '2019-01-16'

# number of epochs，循环次数
num_epoch = round((test_fi - test_ei) / update)
epoch_range = range(0, num_epoch + 1)
timediff = pd.Timedelta(100,unit='d')

In [9]:
def load_lrlist(lr_max,lr_min,num_rounds):
    lrlist = [lr_max+(lr_min-lr_max)*(np.log(i)/np.log(num_rounds)) for i in range(1,num_rounds+1)]
    return lrlist

In [10]:
lr_max = 0.05
lr_min = 0.02
n_jobs = -1

#gbm params
params =  {
    'boosting_type' : 'gbdt',
    'num_leaves' : 31,
    'feature_fraction' : 0.8,
    'bagging_fraction' : 0.8,
    'lambda_l1':1,
    'lambda_l2':10,
    'max_bin' : 64,
    'num_boost_round': 200,
    'learning_rate' : 0.04,
    'min_data_in_leaf':10,
    'num_threads': n_jobs
}
params['objective'] = 'regression'
params['metric'] = {'l2', 'auc'}
lrlist = load_lrlist(lr_max,lr_min,params['num_boost_round'])
params['lrlist'] = lrlist

### get data

In [11]:
def get_data(file_loc, f_x, date, thresh):
    result_list = []
    for name in f_x:
        #print(name)
        alpha = pd.read_csv(file_loc + name + '.csv')
        if alpha.columns.values.tolist()[0] == 'date':
            alpha = alpha.rename(columns={"date": "tradeDate"})
        if file_loc != 'alphas_all/066_':
            alpha = alpha.set_index(['tradeDate']).stack().reset_index().rename(columns={'level_1': 'ticker', 0: name})
        alpha['ticker'] = alpha['ticker'].astype(int)
        if date != '':
            alpha = alpha[alpha.tradeDate == date]
        alpha = alpha.set_index(['ticker', 'tradeDate'])
        alpha = alpha.sort_index(ascending=True)
        result_list.append(alpha)
    result = pd.concat(result_list, axis=1)
    result = result.reset_index()
    result = result.dropna(thresh=thresh)
    return result

In [25]:
f_temp = [item for item in f_x if item not in extra_factors]
print(len(f_temp))
factors = get_data('pct1_cal/每日预测/data/', f_temp, '', 40)
factors = factors[factors.tradeDate >= '2022-07-05']
factors

131


,ticker,tradeDate,minute_FACTORS_RSJ,minute_FACTORS_UID,minute_FACTORS_adj_mom,minute_FACTORS_arpp,minute_FACTORS_kurtosis,minute_FACTORS_skewness,minute_FACTORS_weipan,py_cov_FACTORS_PV_Corr,...,alpha_40,alpha_27,alpha_11,alpha_42,alpha_12,alpha_1,alpha_23,alpha_5,alpha_9,alpha_31
45,1,2022-07-05,-0.941255,0.197885,0.034188,0.484756,8.473642,0.831814,0.058103,1.320486,...,0.511905,0.479590,33.165407,0.511905,0.724747,0.626794,-0.070045,NaN,0.521197,0.860377
46,1,2022-07-06,-1.585938,0.190757,0.018790,0.468865,8.457944,0.836587,0.085166,1.633836,...,0.511905,0.479810,36.442407,0.511905,0.174008,0.672388,NaN,NaN,0.567010,0.871045
47,1,2022-07-07,-0.496620,0.193372,0.005625,0.457350,8.566172,0.861588,0.039798,1.720790,...,0.511905,0.479814,40.326983,0.511905,0.355323,0.685362,0.086308,NaN,0.540063,0.888937
48,1,2022-07-08,-6.214012,0.196697,0.004815,0.465890,8.568543,0.863453,0.076260,1.539228,...,0.511905,0.480017,43.660454,0.511905,0.564133,0.720885,-0.076205,NaN,0.018842,0.901236
49,1,2022-07-11,-2.967066,0.198711,0.015541,0.473107,8.401983,0.847970,0.051880,1.013738,...,0.511905,0.480117,45.982131,0.511905,0.515138,0.750650,-0.016622,NaN,0.617857,0.899870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311767,1914,2022-07-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.503968,0.478519,NaN,NaN,0.375543,0.463299,0.000000,NaN,NaN,0.779844
311768,1914,2022-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.503968,0.478519,NaN,NaN,0.225401,0.496331,-0.007818,NaN,NaN,0.782872
311769,1914,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.503968,0.478627,NaN,NaN,0.253477,0.491459,0.000000,NaN,NaN,0.794728
311770,1914,2022-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.503968,0.477979,NaN,NaN,0.694908,0.564589,-0.027811,NaN,NaN,0.801620


### 7/05 - 8/03

In [13]:
def get_jingjia(filenames):
    result_list = []
    for name in filenames:
        #print(name)
        alpha = pd.read_csv(name)
        alpha = alpha.drop(columns='Unnamed: 0')
        alpha  = alpha.rename(columns={"dataDate": "tradeDate"})
        alpha['ticker'] = alpha['ticker'].astype(int)
        alpha = alpha.set_index(['ticker', 'tradeDate'])
        alpha = alpha.sort_index(ascending=True)
        result_list.append(alpha)
    result = pd.concat(result_list)
    result = result.reset_index()
    return result

In [14]:
filenames = glob.glob('pct1_cal/每日预测/jhjj_data/*.csv')
alpha = get_jingjia(filenames)
alpha = alpha.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
alpha

,ticker,tradeDate,volume,cachgPct,thecommittee,askVolume1,bidVolume1,OCVP1,Gap,Open/vwap-1,caTr,caQrr
0,1,2022-07-05,337600,-0.005366,0.943528,2600,22500.0,0.003027,-0.028815,-0.012518,0.001730,0.679993
1,1,2022-07-06,1766915,0.000688,0.200908,1200,1585.0,0.016935,-0.026122,-0.001991,0.009111,5.233753
2,1,2022-07-07,435094,-0.002751,0.209303,6000,45906.0,0.005959,-0.010239,0.000966,0.002255,0.246245
3,1,2022-07-08,435094,-0.002751,0.209303,6000,45906.0,0.006802,-0.010239,-0.001377,0.002236,0.766280
4,1,2022-07-11,698900,-0.003458,-0.474157,50934,60500.0,0.011426,-0.010981,-0.002423,0.003589,1.606320
...,...,...,...,...,...,...,...,...,...,...,...,...
96971,689009,2022-07-28,4154,0.012338,-0.049363,199,200.0,0.001079,-0.033992,0.000138,0.000591,0.259771
96972,689009,2022-07-29,9357,0.004635,-0.777762,400,643.0,0.004691,-0.031663,-0.009380,0.001321,2.252528
96973,689009,2022-08-01,24173,-0.010491,0.176882,6214,200.0,0.004412,-0.033789,0.020191,0.003362,2.583413
96974,689009,2022-08-02,2418,0.001507,0.764957,389,1382.0,0.000513,-0.030623,-0.006724,0.000340,0.100029


In [15]:
mevtransformer = ExcludeExtreme()
mevtransformer.fit(alpha[extra_factors])
alpha[extra_factors] = mevtransformer.transform(alpha[extra_factors])

In [16]:
# %% a list of classes
## exclude extreme values
class ExcludeExtreme(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_median = None
        self.df_standard = None

    def fit(self, df_columnList):
        self.df_median = df_columnList.median()
        self.df_standard = df_columnList.apply(lambda x: x - self.df_median[x.name]).abs().median()
        return self

    def scaller(self, x):
        self.di_max = self.df_median[x.name] + 5 * self.df_standard[x.name]
        self.di_min = self.df_median[x.name] - 5 * self.df_standard[x.name]
        x = x.apply(lambda v: self.di_min if v < self.di_min else v)
        x = x.apply(lambda v: self.di_max if v > self.di_max else v)
        return x

    def transform(self, df_columnList):
        df_columnList = df_columnList.apply(self.scaller)
        return df_columnList

## fill empty values
class FillEmpty(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.df_mean_industry = None

    def fit(self, df_name, f_x, group_name='industry'):
        self.df_mean_industry = df_name.groupby(group_name).mean()[f_x]
        self.df_mean_industry = self.df_mean_industry.fillna(self.df_mean_industry.mean())
        self.df_mean_industry.columns = [x + '_mean' for x in self.df_mean_industry.columns]
        self.df_mean_industry = self.df_mean_industry.reset_index()
        self.df_mean_industry = self.df_mean_industry.fillna(0)  # 可以删除
        return self

    def transform(self, df_name, f_x, group_name='industry'):
        df_name_mean = df_name.merge(self.df_mean_industry, on=group_name, how='left')
        df_name_mean[f_x] = df_name_mean[f_x].apply(lambda x: x.fillna(df_name_mean[x.name + '_mean']))
        df_name = df_name_mean[df_name.columns]
        return df_name

## neutralize industry
class Neutralization(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.models = {}

    def regr_fit(self, X, y):
        self.models[y.name] = linear_model.LinearRegression().fit(X, y)
        return

    def regr_pred(self, X, y):
        pred = self.models[y.name].predict(X)
        return y - pred

    def fit(self, df_name, f_x, f_idst = f_industry):
        X = df_name[['log_marketValue'] + f_idst]
        df_name[f_x].apply(lambda y: self.regr_fit(X, y))
        return self

    def transform(self, df_name, f_x, f_idst = f_industry):
        X = df_name[['log_marketValue'] + f_idst]
        df_name[f_x] = df_name[f_x].apply(lambda y: self.regr_pred(X, y))
        return df_name

In [17]:
strData = ticekrToStr(factors[['ticker']])
tickers = strData.ticker.unique()
tickers = tickers.tolist()

In [18]:
dates = factors.tradeDate.sort_values().unique()
timediff = pd.Timedelta(100,unit='d')

In [19]:
def get_target(tickers):
    outData = DataAPI.MktEqudGet(ticker=tickers, beginDate='2022-07-05', endDate='2022-08-03', pandas="1")
    outData['ticker'] = outData['ticker'].astype(int)
    outData = outData.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
    outData['log_marketValue'] = outData.marketValue.apply(np.log)
    return outData[['ticker', 'tradeDate', 'log_marketValue']]

In [26]:
marketValue = get_target(tickers)
factors = factors.merge(marketValue, on=['ticker', 'tradeDate'], how='left')

In [27]:
industry_data = DataAPI.EquIndustryGet(ticker=tickers, pandas="1")
industry_name = industry_data.groupby('ticker').head(1)[['ticker','industryName1']].reset_index(drop =True)
industry_name = industry_name.rename(columns={"industryName1": "industry"})
industry_name['ticker'] = industry_name['ticker'].astype(np.int64)
factors = factors.merge(industry_name, on=['ticker'], how='left')

In [28]:
factors.industry = factors.industry.fillna('其他')
industry_to_number = {}
for i, v in enumerate(factors.industry.sort_values().unique()):
    industry_to_number[v] = i+1
print(industry_to_number)
factors.industry = factors.industry.map(industry_to_number)
df_idst = pd.get_dummies(factors.industry, prefix='idst')
factors = factors.merge(df_idst, left_index=True, right_index=True)
display(factors)

{'交通运输': 1, '传媒': 2, '农林牧渔': 3, '医药': 4, '商贸零售': 5, '国防军工': 6, '基础化工': 7, '家电': 8, '建材': 9, '建筑': 10, '房地产': 11, '有色金属': 12, '机械': 13, '汽车': 14, '消费者服务': 15, '煤炭': 16, '电力及公用事业': 17, '电力设备及新能源': 18, '电子': 19, '石油石化': 20, '纺织服装': 21, '综合': 22, '计算机': 23, '轻工制造': 24, '通信': 25, '钢铁': 26, '银行': 27, '非银行金融': 28, '食品饮料': 29}


,ticker,tradeDate,minute_FACTORS_RSJ,minute_FACTORS_UID,minute_FACTORS_adj_mom,minute_FACTORS_arpp,minute_FACTORS_kurtosis,minute_FACTORS_skewness,minute_FACTORS_weipan,py_cov_FACTORS_PV_Corr,...,idst_20,idst_21,idst_22,idst_23,idst_24,idst_25,idst_26,idst_27,idst_28,idst_29
0,1,2022-07-05,-0.941255,0.197885,0.034188,0.484756,8.473642,0.831814,0.058103,1.320486,...,0,0,0,0,0,0,0,1,0,0
1,1,2022-07-06,-1.585938,0.190757,0.018790,0.468865,8.457944,0.836587,0.085166,1.633836,...,0,0,0,0,0,0,0,1,0,0
2,1,2022-07-07,-0.496620,0.193372,0.005625,0.457350,8.566172,0.861588,0.039798,1.720790,...,0,0,0,0,0,0,0,1,0,0
3,1,2022-07-08,-6.214012,0.196697,0.004815,0.465890,8.568543,0.863453,0.076260,1.539228,...,0,0,0,0,0,0,0,1,0,0
4,1,2022-07-11,-2.967066,0.198711,0.015541,0.473107,8.401983,0.847970,0.051880,1.013738,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101833,1914,2022-07-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
101834,1914,2022-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
101835,1914,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
101836,1914,2022-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [45]:
def run_df(f_temp, date):
#for date in dates:
    print('----- DATE {}------'.format(date))
    df2 = factors[factors.tradeDate == date]
    print('处理前无效值：', df2['log_marketValue'].isnull().any().sum())
    df2 = df2[~df2.log_marketValue.isnull()]
    print('处理后无效值：', df2['log_marketValue'].isnull().any().sum())

    # 中位数去极值
    print('exclude extreme values')
    mevtransformer = ExcludeExtreme()
    mevtransformer.fit(df2[f_temp])
    df2[f_temp] = mevtransformer.transform(df2[f_temp])

    # 缺失值处理
    print('deal with null values')
    print('缺失值：', df2[f_temp].isnull().any().sum())
    gvfiller = FillEmpty()
    gvfiller = gvfiller.fit(df2, f_temp)
    df2 = gvfiller.transform(df2, f_temp)
    print('缺失值：', df2[f_temp].isnull().any().sum())
    df2[f_temp] = df2[f_temp].fillna(0)

    # 行业中性处理
    print('do neutralization')
    idst_neutral = Neutralization()
    idst_neutral = idst_neutral.fit(df2, f_temp)
    df2 = idst_neutral.transform(df2, f_temp)

    # 标准化处理
    print('do standardization')
    scaler = StandardScaler()
    df2[f_temp] = scaler.fit_transform(df2[f_temp])

    df2 = df2[f_index + f_temp]
    return df2

print('PARALLEL')
start = time.time()
parallel_obj = Parallel(n_jobs=-1)(delayed(run_df)(f_temp, date) for date in dates)
print(f'耗时:{time.time() - start}')
test_data = pd.concat(parallel_obj)
print('sort values')
test_data = test_data.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
print('reduce memory')
test_data = reduce_mem_usage(test_data)
print('store data')

PARALLEL
----- DATE 2022-07-25------
处理前无效值： 0
处理后无效值： 0
exclude extreme values
----- DATE 2022-07-06------
处理前无效值： 0
处理后无效值： 0
exclude extreme values
----- DATE 2022-07-05------
----- DATE 2022-07-12------
----- DATE 2022-07-11------
----- DATE 2022-07-08------
处理前无效值： 0
处理后无效值： 0
exclude extreme values
----- DATE 2022-07-07------
处理前无效值： 0
处理后无效值： 0
exclude extreme values
处理前无效值：处理前无效值： 0
 0
处理后无效值： 0
exclude extreme values
处理后无效值： 0
exclude extreme values
处理前无效值： 0
处理后无效值： 0
exclude extreme values
----- DATE 2022-07-14------
----- DATE 2022-07-15------
----- DATE 2022-07-13------
处理前无效值： 0
处理后无效值： 0
exclude extreme values
----- DATE 2022-07-18------
处理前无效值： 0
处理后无效值： 0
exclude extreme values
处理前无效值： 0
处理后无效值： 0
exclude extreme values
处理前无效值： 0
处理后无效值： 0
exclude extreme values
----- DATE 2022-07-20------
处理前无效值： 0
处理后无效值： 0
exclude extreme values
----- DATE 2022-07-21------
----- DATE 2022-07-22------
处理前无效值： 0
处理后无效值： 0
exclude extreme values
处理前无效值： 0
处理后无效值： 0
exclude extreme valu

In [46]:
test_data = test_data.merge(alpha, on=['ticker', 'tradeDate'], how='left')
test_data.to_csv('pct1_cal/每日预测/OneMonthData.csv')
test_data

,ticker,tradeDate,minute_FACTORS_RSJ,minute_FACTORS_UID,minute_FACTORS_adj_mom,minute_FACTORS_arpp,minute_FACTORS_kurtosis,minute_FACTORS_skewness,minute_FACTORS_weipan,py_cov_FACTORS_PV_Corr,...,volume,cachgPct,thecommittee,askVolume1,bidVolume1,OCVP1,Gap,Open/vwap-1,caTr,caQrr
0,1,2022-07-05,-0.715332,-0.569336,-0.278320,0.582520,-1.028320,-0.953613,0.225708,0.967773,...,152367.5,-0.005366,0.943528,2600.0,18300.0,0.003027,-0.028815,-0.012518,0.001730,0.679993
1,1,2022-07-06,-1.655273,-0.619141,-0.302734,0.239258,-1.053711,-0.975098,0.849609,1.156250,...,152367.5,0.000688,0.200908,1200.0,1585.0,0.013767,-0.026122,-0.001991,0.009111,4.039546
2,1,2022-07-07,-0.365723,-0.580566,-0.444580,-0.116699,-1.048828,-0.951172,-0.752441,1.260742,...,152367.5,-0.002751,0.209303,6000.0,18300.0,0.005959,-0.010239,0.000966,0.002255,0.246245
3,1,2022-07-08,-1.711914,-0.624023,-0.220337,0.056641,-0.843262,-0.754395,0.530273,1.357422,...,152367.5,-0.002751,0.209303,6000.0,18300.0,0.006802,-0.010239,-0.001377,0.002236,0.766280
4,1,2022-07-11,-2.265625,-0.917969,-0.189331,0.045807,-0.981934,-0.882324,-0.509766,1.113281,...,152367.5,-0.003458,-0.474157,15200.0,18300.0,0.011426,-0.010981,-0.002423,0.003589,1.606320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101833,689009,2022-07-28,0.771484,-0.890625,-0.087585,0.763672,-1.111328,-1.359375,0.526855,-0.304199,...,4154.0,0.012338,-0.049363,199.0,200.0,0.001079,-0.033992,0.000138,0.000591,0.259771
101834,689009,2022-07-29,0.763184,-0.982422,-0.635742,0.685059,-1.095703,-1.270508,-0.882812,-0.377197,...,9357.0,0.004635,-0.777762,400.0,643.0,0.004691,-0.031663,-0.009380,0.001321,2.252528
101835,689009,2022-08-01,0.565918,0.331787,0.642090,0.800781,-1.087891,-1.261719,0.036102,0.103149,...,24173.0,-0.010491,0.176882,6214.0,200.0,0.004412,-0.033789,0.020191,0.003362,2.583413
101836,689009,2022-08-02,0.360107,2.654297,0.550781,0.965820,-1.049805,-1.191406,-0.764648,0.229614,...,2418.0,0.001507,0.764957,389.0,1382.0,0.000513,-0.030623,-0.006724,0.000340,0.100029


In [51]:
for f_y in f_y_list:
    result_name = 'Month_{}_{}_{}_{}'.format(data_source, model_name, f_y, target_type)
    print(result_name)

    df_result_all = pd.DataFrame()
    total_IC = 0
    start = time.time()
    epoch_range = range(0, 1)

    for epoch in epoch_range:
        epoch = num_epoch + 1
        print('----- EPOCH {}------'.format(epoch))
        update_n = epoch * update
        # get a list of train dates
        epoch_t_train = epoch_ts[train_si + update_n : train_ei + update_n]
        #print('训练时间: ', epoch_t_train)
        df_train = df1[df1.tradeDate.apply(lambda x: x in epoch_t_train)].reset_index(drop=True)
        print('数据大小: ', df_train.shape)

        # 获得 x
        x_train = df_train[f_x].values
        print('处理后x:', x_train.shape)

        # 获得y
        y_train = df_train[f_y].values
        print('处理后y:', y_train.shape)
        lgb_train = lgb.Dataset(x_train, y_train)
        model = lgb.train(params,
                        lgb_train,
                        num_boost_round=params['num_boost_round'],
                        callbacks = [lgb.reset_parameter(learning_rate = params['lrlist'])]
                        )
        y_pred = model.predict(test_data[f_x])

        # 获得结果
        print(f'耗时:{time.time() - start}')
        print('get result')
        df_result = test_data[f_index].copy()
        df_result['y_pred'] = y_pred
        df_result_all = df_result_all.append(df_result)

    print(f'耗时:{time.time() - start}') 
    print('sort values')
    df_result_all = df_result_all.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
    today = (datetime.datetime.now()).strftime("%Y-%m-%d:%H:%M:%S")
    df_result_all.to_csv('{}/{}/{}_{}.csv'.format(file_location, file_type, result_name, today), index=False)
    print('======== COMPLETED {} ========'.format(model_name))


Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_rank_r
----- EPOCH 36------
数据大小:  (1972387, 308)
处理后x: (1972387, 141)
处理后y: (1972387,)


/home/liufengyuan/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.241152 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8896
[LightGBM] [Info] Number of data points in the train set: 1972387, number of used features: 139
[LightGBM] [Info] Start training from score 0.499733
耗时:434.11669278144836
get result
耗时:434.16792583465576
sort values
======== COMPLETED LGBMRegressor ========
Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_r
----- EPOCH 36------
数据大小:  (1972387, 308)
处理后x: (1972387, 141)
处理后y: (1972387,)


/home/liufengyuan/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.257524 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8896
[LightGBM] [Info] Number of data points in the train set: 1972387, number of used features: 139
[LightGBM] [Info] Start training from score -0.000068
耗时:335.026682138443
get result
耗时:335.0653717517853
sort values
======== COMPLETED LGBMRegressor ========
Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_full_r
----- EPOCH 36------
数据大小:  (1972387, 308)
处理后x: (1972387, 141)
处理后y: (1972387,)


/home/liufengyuan/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.232823 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8896
[LightGBM] [Info] Number of data points in the train set: 1972387, number of used features: 139
[LightGBM] [Info] Start training from score 0.000334
耗时:210.9994957447052
get result
耗时:211.04180884361267
sort values
======== COMPLETED LGBMRegressor ========
Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_full_rank_r
----- EPOCH 36------
数据大小:  (1972387, 308)
处理后x: (1972387, 141)
处理后y: (1972387,)


/home/liufengyuan/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Unknown parameter: lrlist
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.713194 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8896
[LightGBM] [Info] Number of data points in the train set: 1972387, number of used features: 139
[LightGBM] [Info] Start training from score 0.499244
耗时:277.9963867664337
get result
耗时:278.0612061023712
sort values
======== COMPLETED LGBMRegressor ========


In [2]:
def convertFile(filenames, file_location):
    for file in filenames:
        name = file[:-26]
        print(name)
        df = pd.read_csv(file_location + file)
        df.rename(columns = {'tradeDate': 'date', 'y_pred': 'prediction'}, inplace = True)
        df['date'] = df.date.apply(lambda x: str(x).replace('-', ''))
        df = df[['ticker', 'date', 'prediction']]

        dates = df.date.sort_values().unique()
        def run(date):
            print(date)
            temp = df[df.date == date]
            temp = temp.sort_values(by=['prediction'], ascending= False).reset_index(drop=True)
            #temp.to_csv('Predictions/zz500_{}/{}.csv'.format(name, date), index=False)
            temp.to_csv('{}{}/{}.csv'.format(file_location, name, date), index=False)

        print('PARALLEL')
        start = time.time()
        parallel_obj = Parallel(n_jobs=-1)(delayed(run)(date) for date in dates)
        print(f'耗时:{time.time() - start}')

In [3]:
filenames = ['Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_full_r_2022-08-04:21:28:12.csv',
'Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_full_rank_r_2022-08-04:21:32:50.csv',
'Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_r_2022-08-04:21:24:41.csv',
'Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_rank_r_2022-08-04:21:19:05.csv']
file_location = 'pct1_cal/full/'
convertFile(filenames, file_location)

Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_full
PARALLEL
20220705
20220707
20220706
20220708
20220711
20220712
20220713
20220714
20220715
20220720
20220718
20220719
20220725
20220721
20220726
20220722
20220727
20220728
20220729
20220803
20220801
20220802
耗时:2.518691301345825
Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_full_rank
PARALLEL
20220705
20220708
20220706
20220707
20220711
20220712
20220713
20220714
20220715
20220718
20220719
20220722
20220720
20220725
20220721
20220726
20220727
20220801
20220728
20220802
20220729
20220803
耗时:0.9520299434661865
Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1
PARALLEL
20220705
20220713
20220706
20220707
20220719
20220708
20220712
20220711
20220714
20220718
20220715
20220727
20220729
20220720
20220725
20220721
20220722
20220726
20220801
20220802
20220728
20220803
耗时:0.9220516681671143
Month_full_Alter_066_modified_exta_LGBMRegressor_openclose_pct1_rank
PARALLEL
20220705
20220707
20220711
20